|      |      | 
| ---- | ---- | 
| Journal:|Sustainability (ISSN 2071-1050)|
|Manuscript ID:| sustainability-1485080|
|Type:|Article|
|Title:|NEVs Supply Chain Coordination with Financial Constraint and Demand Uncertainty|
|manuscripts:|P.9 - figure 9. Data generation code|
|Code authors:|Yongjian Li$^*$ |
|Last modify:|2021-01-12|
|version:|Python 3.6.5, sympy >= 1.3|

In [1]:
from sympy import *
from sympy.stats import *
import numpy as np
import pandas as pd
# init_printing(use_unicode=True)
import sympy
sympy.__version__

'1.9'

In [2]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

In [3]:
# Initial parameter values
parameter = {p: 10, c: 4, s: 3, w: 6, theta: 0.8, b: 3, B: 100, r: 0.05,
             mu: 100, sigma: 100}

#### (see manuscript P(9) figure 9.)


\begin{align}
&F^{-1}(\alpha) \\
&\bar{B}=(c(1+r)-s) F^{-1}\left(\frac{p-c(1+r)}{p-s}\right)-(p-s) F^{-1}(\alpha)
\end{align}




- Suppose X is normally distributed with $\mu=100$,$\sigma=100$ 
- CDF,exp1: $F(x) = \alpha$

In [4]:
X = Normal(x, mu, sigma).subs(parameter)
exp1 = Eq(cdf(X)(x), alpha)
exp1

Eq(erf(sqrt(2)*(x - 100)/200)/2 + 1/2, alpha)

- inverse exp1: $F^{-1}(\alpha) = x ,\quad \{0\leqslant \alpha \leqslant 1\}$

In [5]:
i_cdf = solve(exp1, x)[0]
i_cdf

100*sqrt(2)*erfinv(2*alpha - 1) + 100

In [6]:
# Convert a SymPy expression into a function that allows for fast numeric evaluation.
f = lambdify(alpha, i_cdf, modules=['numpy', 'sympy'])

x_0 = [i/1000 for i in range(1, 1000)]

dt = {
    "x_0": x_0,
}

df = pd.DataFrame(dt)
df['f(x)'] = df['x_0'].map(f)
df1 = df.copy()
df1['f(x)'] = df1['f(x)'].map(int)
df1 = df1.drop_duplicates(subset=['f(x)']).reset_index(drop=True).copy()
df1

,x_0,f(x)
0,0.001,-209
1,0.002,-187
2,0.003,-174
3,0.004,-165
4,0.005,-157
...,...,...
422,0.995,357
423,0.996,365
424,0.997,374
425,0.998,387


In [7]:
df1.query("`f(x)`==0")

,x_0,f(x)
113,0.157,0


In [8]:
# df1.iloc[::10].to_csv("inverse(CDF)",index=None,sep='\t')

$$\bar{B}=(c(1+r)-s) F^{-1}\left(\frac{p-c(1+r)}{p-s}\right)-(p-s) F^{-1}(\alpha)$$

In [11]:
prob = (p-c *(1+r))/(p - s)
prob

(-c*(r + 1) + p)/(p - s)

In [13]:
prob = prob.subs(parameter)
eq = Eq(cdf(X)(x), prob)
qs = solve(eq, x)[0].evalf(subs=parameter)
qs =int(qs)
qs

194

In [33]:
expr1 = (c*(1+r)-s)*194.853504082664 - (p-s) * x
expr1

194.853504082664*c*(r + 1) - 194.853504082664*s - x*(p - s)

In [34]:
f2 = lambdify(x, expr1.subs(parameter), modules=['numpy', 'sympy'])

piqc = df.copy()
piqc['f(x)'] = piqc['f(x)'].map(float)
piqc['f(x)'] = piqc['f(x)'].map(f2)
piqc


,x_0,f(x)
0,0.001,1696.986819
1,0.002,1548.537422
2,0.003,1457.271175
3,0.004,1390.273070
4,0.005,1336.904717
...,...,...
994,0.995,-2269.256308
995,0.996,-2322.624661
996,0.997,-2389.622765
997,0.998,-2480.889012


In [35]:
piqc.query("x_0 == 0.252")

,x_0,f(x)
251,0.252,1.570715


In [37]:
piqc.iloc[::5].to_csv("Bbar",index=None,sep='\t')

- some help fun() note here.

In [5]:
## note here
## Given a value, calculate the cumulative probability
value = 0
cdf(X)(value).evalf()
## Given a value, calculate the cumulative probability
value = 189.380
P(X < value).evalf(subs=parameter)